# DCGAN (Deep Convolutional GAN)

In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim

from torchvision import transforms, datasets
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

import numpy as np


In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),             # Converts PIL image to tensor
    transforms.Normalize([0.5], [0.5]) # Normalize pixel values from [0,1] to [-1,1]
])


In [ ]:
dataloader = DataLoader(
    datasets.MNIST('.', train=True, download=True, transform=transform),
    batch_size=128,
    shuffle=True
)

#image size: 28x28


Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9912422/9912422 [00:02<00:00, 4501090.28it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28881/28881 [00:00<00:00, 256471.17it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1648877/1648877 [00:00<00:00, 1737391.82it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4542/4542 [00:00<00:00, 683206.45it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [12]:
# Generator (for MNIST - 28x28 output)
class Generator(nn.Module):
    def __init__(self, z_dim=100, img_channels=1, feature_g=64):
        super().__init__()
        self.net = nn.Sequential(
            #                (inputDim, OutputChannel, kernelSize, stride, padding)
            # nn.ConvTranspose2d(100, 256, 3, 1, 0)
            nn.ConvTranspose2d(z_dim, feature_g * 4, 3, 1, 0),     # 1x1 → 3x3
            nn.BatchNorm2d(feature_g * 4),
            nn.ReLU(True),

            #OutputSize = (7-1)* 2 + 4-2*1 = 14

            # nn.ConvTranspose2d(256, 128, 4,2,1)
            nn.ConvTranspose2d(feature_g * 4, feature_g * 2, 4, 2, 1),  # 3x3 → 7x7
            nn.BatchNorm2d(feature_g * 2),
            nn.ReLU(True),

            # nn.ConvTranspose2d(128, 64, 4,2,1)
            nn.ConvTranspose2d(feature_g * 2, feature_g, 4, 2, 1),      # 7x7 → 14x14  
            nn.BatchNorm2d(feature_g),
            nn.ReLU(True),
            # nn.ConvTranspose2d(64, 1, 4,2,1)
            nn.ConvTranspose2d(feature_g, img_channels, 4, 2, 1),       # 14x14 → 28x28
            nn.Tanh()
        )

    def forward(self, z):
        return self.net(z)

# Discriminator (for MNIST - 28x28 input)
class Discriminator(nn.Module):
    def __init__(self, img_channels=1, feature_d=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(img_channels, feature_d, 4, 2, 1),       # 28x28 → 14x14
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(feature_d, feature_d * 2, 4, 2, 1),       # 14x14 → 7x7
            nn.BatchNorm2d(feature_d * 2),
            nn.LeakyReLU(0.2, inplace=True),

            # nn.Conv2d(feature_d * 2, 1, 7, 1, 0),               # 7x7 → 1x1
            nn.Conv2d(feature_d * 2, 1, 3, 1, 0),               # 7x7 → 1x1
            nn.Sigmoid()
        )

    def forward(self, x):
        # return self.net(x).view(-1, 1)
        return self.net(x).mean([2, 3])


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
noise_dim = 100                    # Size of the noise vector input to Generator

G = Generator(noise_dim).to(device)

D = Discriminator().to(device)

In [15]:
criterion = nn.BCELoss()          # Binary Cross-Entropy for classification
optimizer_G = optim.Adam(G.parameters(), lr=0.0002)
optimizer_D = optim.Adam(D.parameters(), lr=0.0002)

In [ ]:
epochs = 50

#Loop over each batch and epoch.
#_ ignores the labels since GANs are unsupervised.

for epoch in range(epochs):
    for real_images, _ in dataloader:
        real_images = real_images.to(device)
        batch_size = real_images.size(0)

        # Real and fake labels
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # Train on real images
        optimizer_D.zero_grad()
        outputs_real = D(real_images)           # Real image score
        loss_real = criterion(outputs_real, real_labels)

        # Generate fake images
        # z = torch.randn(batch_size, noise_dim).to(device)
        z = torch.randn(batch_size, noise_dim, 1, 1).to(device)
        fake_images = G(z)

        outputs_fake = D(fake_images.detach())  # Detach to avoid updating G
        loss_fake = criterion(outputs_fake, fake_labels)

        # Total discriminator loss
        loss_D = loss_real + loss_fake
        loss_D.backward()
        optimizer_D.step()
        
        optimizer_G.zero_grad()

        outputs = D(fake_images)                # Try to fool D with fake images
        loss_G = criterion(outputs, real_labels)  # G wants D to label them as real (1)
        loss_G.backward()
        optimizer_G.step()
        
        print(f"Epoch [{epoch+1}/{epochs}], Loss D: {loss_D.item():.4f}, Loss G: {loss_G.item():.4f}")

        if (epoch + 1) % 10 == 0:
          with torch.no_grad():  # Turn off gradients for inference
            z = torch.randn(64, noise_dim).to(device)
            fake = G(z).cpu()
            grid = fake.view(64, 1, 28, 28).detach().numpy()

            # Create a 8x8 grid of images
            fig, axs = plt.subplots(8, 8, figsize=(8, 8))
            for i in range(8):
                for j in range(8):
                    axs[i, j].imshow(grid[i*8+j][0], cmap='gray')
                    axs[i, j].axis('off')
            plt.show()



Epoch [1/50], Loss D: 1.4467, Loss G: 0.7969
Epoch [1/50], Loss D: 1.5065, Loss G: 0.6771
Epoch [1/50], Loss D: 1.4965, Loss G: 0.6563
Epoch [1/50], Loss D: 1.4436, Loss G: 0.6788
Epoch [1/50], Loss D: 1.3824, Loss G: 0.7158
Epoch [1/50], Loss D: 1.3210, Loss G: 0.7572
Epoch [1/50], Loss D: 1.2508, Loss G: 0.8124
Epoch [1/50], Loss D: 1.1793, Loss G: 0.8772
Epoch [1/50], Loss D: 1.1165, Loss G: 0.9366
Epoch [1/50], Loss D: 1.0560, Loss G: 0.9940
Epoch [1/50], Loss D: 1.0228, Loss G: 1.0291
Epoch [1/50], Loss D: 0.9807, Loss G: 1.0623
Epoch [1/50], Loss D: 0.9737, Loss G: 1.0675
Epoch [1/50], Loss D: 0.9689, Loss G: 1.0528
Epoch [1/50], Loss D: 0.9665, Loss G: 1.0413
Epoch [1/50], Loss D: 0.9638, Loss G: 1.0399
Epoch [1/50], Loss D: 0.9620, Loss G: 1.0357
Epoch [1/50], Loss D: 0.9372, Loss G: 1.0705
Epoch [1/50], Loss D: 0.9061, Loss G: 1.1123
Epoch [1/50], Loss D: 0.8866, Loss G: 1.1386
Epoch [1/50], Loss D: 0.8720, Loss G: 1.1709
Epoch [1/50], Loss D: 0.8434, Loss G: 1.2007
Epoch [1/5

KeyboardInterrupt: 